# Calculate plasma parameters

In [28]:
%matplotlib inline

In [29]:
import ipywidgets as widgets
from IPython.display import display
from plasmapy.particles import Particle
from astropy.constants.si import m_p, m_e
import astropy.units as units
from inspect import signature
import importlib
import json

In [30]:
values_container = dict()
def create_label(label,color="black"):
    return widgets.HTML(f"<h3 style='margin:0px;color:{color}'>{label}<h3>")
class GenericWidget:
    def __init__(self, property_name,property_alias="",values_cont=values_container):
        self.property_name = property_name
        self.property_alias = property_alias or property_name
        self.widget = None
        self.values_cont = values_cont
        self.unit = None
        self.units_dropdown = None
    def set_unit(self,unit):
        self.unit = unit
    def get_widget(self):
        return self.widget
    def get_dropdown_widget(self):
        return self.units_dropdown
    def set_place_holder(self,text):
        self.widget.placeholder = text
    def create_widget(self):
        raise NotImplementedError()
    def post_creation(self):
        self.set_place_holder("")
        self.widget.observe(self.handle_change,names="value")
    def edge_case(self,value):
        pass
    def edge_case_condition(self,value):
        return False
    def try_change_value(self,value):
        self.values_cont[self.property_name]=value
    def display_error(self,value):
        if self.widget:
            self.widget.layout.border="2px solid red"
            self.widget.description = "Invalid "+self.property_alias
            self.values_cont[self.property_name] = None
    def convert_to_unit(self,change):
        if self.unit:
            return change.new*self.unit
        return change.new
    def attach_units_dropdown(self,options):
        self.units_dropdown = widgets.Dropdown(
            options=options,
            value=options[0],
            layout=widgets.Layout(width="100px")
        )
        self.units_dropdown.observe(self.handle_dropdown_change,names="value")
    def handle_dropdown_change(self,change):
        self.set_unit(self.units_dropdown.value)
        if self.property_name in self.values_cont:
            self.values_cont[self.property_name] = self.values_cont[self.property_name].value*self.unit
    def handle_change(self,change):
        value = self.convert_to_unit(change)
        if self.edge_case_condition(value):
            self.edge_case(value)
        else:
            try:
                self.try_change_value(value)
            except:
                self.display_error(value)

class FloatBox(GenericWidget):
    def __init__(self, property_name,min=-1e50,max=1e50):
        super().__init__(property_name)
        self.min = min
        self.max = max
    def create_widget(self,style={"description_width": "initial"}):
        self.widget = widgets.BoundedFloatText(name=self.property_name,min=self.min, 
            max=self.max, value=0, step=0.1,style=style)
        self.post_creation()

class CheckBox(GenericWidget):
    def __init__(self, property_name):
        super().__init__(property_name)
    def create_widget(self):
        self.widget = widgets.Checkbox(value=False)
        self.post_creation()
    def try_change_value(self,value):
        self.values_cont[self.property_name]=value

class ParticleBox(GenericWidget):
    def __init__(self, property_name,property_alias=None):
        super().__init__(property_name,property_alias=property_alias)

    def edge_case_condition(self, value):
        return value is None or value == ""
    def edge_case(self, value):
        self.values_cont[self.property_name] = None
        # self.values_cont["ion"] = None
        self.widget.description = ""
        self.widget.layout.border = "1px solid black"
    def try_change_value(self, value):
        particle = Particle(value)
        # if ion.is_ion:
        #     self.values_cont["ion"] = ion
        self.values_cont[self.property_name] = particle
        self.widget.layout.border="1px solid black"
        self.widget.description = ""

    def create_widget(self,style={"description_width": "initial"}):
        self.widget = widgets.Text(style=style)
        self.post_creation()

class IonBox(ParticleBox):
    def __init__(self, property_name,property_alias=None):
        super().__init__(property_name,property_alias=property_alias)
    def try_change_value(self, value):
        ion = Particle(value)
        if ion.is_ion:
            self.values_cont[self.property_name] = ion
            self.widget.layout.border="1px solid black"
            self.widget.description = ""
        else:
            raise ValueError(f"{ion} is not an ion")

def create_widget(widget_type,**kwargs):
    unit = None
    placeholder = None
    opts=None
    if 'unit' in kwargs:
        unit = kwargs['unit']
        del kwargs['unit']
    if 'placeholder' in kwargs:
        placeholder = kwargs['placeholder']
        del kwargs['placeholder']
    if 'opts' in kwargs:
        opts = kwargs['opts']
        del kwargs['opts']
    widget_element = widget_type(**kwargs)
    widget_element.create_widget()
    if unit:
        widget_element.set_unit(unit)
    if placeholder:
        widget_element.set_place_holder(placeholder)
    if opts:
        widget_element.attach_units_dropdown(opts)
        return [widget_element.get_widget(),widget_element.get_dropdown_widget()]
    return widget_element.get_widget()

In [31]:
def colored(r, g, b, text):
    return "\033[38;2;{};{};{}m{} \033[38;2;255;255;255m".format(r, g, b, text)
class FunctionInfo:
    def __init__(self, module_name, function_name, values_cont=values_container):
        self.module = module_name
        self.fname = function_name
        self.fattr = getattr(importlib.import_module(module_name),function_name)
        self.values_cont = values_cont
        self.spec_combo = None
        self.sig = list(signature(self.fattr).parameters.keys())
        self.output_widget = widgets.Output()
        self.output_widget.layout.margin = "10px 10px 10px 10px"
        self.output_widget.layout.padding = "10px 10px 10px 10px"
    def add_combo(self,spec_combo):
        if not self.spec_combo:
            self.spec_combo = []
        self.spec_combo.append(spec_combo)
    def get_output_widget(self):
        return self.output_widget
    def produce_arg(self, spec):
        args_dict = dict()
        for arg in spec:
            if arg in self.values_cont and self.values_cont[arg] is not None:
                args_dict[arg] = self.values_cont[arg]

        return args_dict

    def error_message(self,spec):
        print(colored(0,0,0,"["),end="")
        for arg in spec:
            if arg in self.values_cont and self.values_cont[arg] is not None:
                print(colored(0,128,0,arg+":present,"),end="")
            else:
                print(colored(255,0,0,arg+":missing,"),end="")
        print(colored(0,0,0,"]"))
    def process(self):
        self.output_widget.clear_output()
        args_dict = dict()
        if self.spec_combo:
           for spec in self.spec_combo:
               args_dict = self.produce_arg(spec)
               
               if len(args_dict) == len(spec):
                   break
        else:
            args_dict = self.produce_arg(self.sig)
        with self.output_widget:
            try:
                self.output_widget.layout.border="0px"
                print(" : "+str(self.fattr(**args_dict)))
            except Exception as e:
                self.output_widget.layout.border="1px solid red"
                print(e)
                print(" : could not be computed one or more parameter is missing - check below for missing parameters")
                if self.spec_combo:
                    for spec in self.spec_combo:
                        self.error_message(spec)
                else:
                    self.error_message(self.sig)


In [32]:
with open('properties_metadata.json') as f:
    data = json.load(f)

process_queue = []
app = widgets.Tab()
children = []
for i,title in enumerate(data):
    grid_layout = widgets.GridspecLayout(10,2,width="100%")
    for j,prop in enumerate(data[title]):
        fn = FunctionInfo(prop["module_name"],prop["function_name"])
        if "spec_combo" in prop:
            for spec_combo in prop["spec_combo"]:
                fn.add_combo(spec_combo)
        grid_layout[j,0] = create_label(prop["function_name"]+":")
        grid_layout[j,1] = fn.get_output_widget()
        process_queue.append(fn)
    children.append(grid_layout)
    app.set_title(i,title)
app.children = children

In [33]:
def create_button():
    button = widgets.Button(description="Calculate Properties",button_style="info")
    return button

def handle_button_click(event):
    for fn in process_queue:
        fn.process()

In [34]:
## grid config
grid = widgets.GridspecLayout(15, 3)
grid.layout.margin="10px"

# grid_data = [
#     [
#         create_label("Parameter",color="#00BFD8"),
#         create_label("Value",color="#00BFD8"),
#         create_label("Unit",color="#00BFD8")
#     ],
#     [
#         create_label("B - Magnetic Field Magnitude:"),
#         *create_widget(FloatBox,property_name="B",unit=units.T,opts=[units.T,units.G,units.uG])
#     ],
#     [
#         create_label("Particle:"),
#         create_widget(ParticleBox,property_name="particle",property_alias="particle_type",placeholder="Enter particle e.g. neutron,He")
#     ],
#     [
#         create_label("Ion:"),
#         create_widget(IonBox,property_name="ion",property_alias="ion_type",placeholder="Enter ion e.g. He 2+")
#     ],
#     [
#         create_label("Convert to Hertz:"),
#         create_widget(CheckBox,property_name="to_hz")
#     ],
#     [
#         create_label("_"*20,color="#00BFD8"),
#         create_label("Density Number"),
#         create_label("_"*20,color="#00BFD8")
#     ],
#     [
#         create_label("n - Standard Density Number:"),
#         *create_widget(FloatBox,property_name="n",unit=units.m**-3,
#             opts=[units.m**-3,units.cm**-3,units.mm**-3])
#     ],
#     [
#         create_label("n<sub>e</sub> - Electron Density Number:"),
#         *create_widget(FloatBox,property_name="n_e",unit=units.m**-3,
#             opts=[units.m**-3,units.cm**-3,units.mm**-3])
#     ],
#     [
#         create_label("n<sub>i</sub> - Ion Density Number:"),
#         *create_widget(FloatBox,property_name="n_i",unit=units.m**-3,
#             opts=[units.m**-3,units.cm**-3,units.mm**-3])
#     ],
#     [
#         create_label("_"*20,color="#00BFD8"),
#         create_label("Temperature"),
#         create_label("_"*20,color="#00BFD8")
#     ],
#     [
#         create_label("T - Standard Temperature:"),
#         create_widget(FloatBox,property_name="T",min=0,unit=units.K),
#         create_label("K",color="#a9a9a9")
#     ],
#     [
#         create_label("T<sub>e</sub> - Electron Temperature:"),
#         create_widget(FloatBox,property_name="T_e",min=0,unit=units.K),
#         create_label("K",color="#a9a9a9")
#     ],
#     [
#         create_label("T<sub>i</sub> - Ion Temperature:"),
#         create_widget(FloatBox,property_name="T_i",min=0,unit=units.K),
#         create_label("K",color="#a9a9a9")
#     ],
    
# ]


# for i,row in enumerate(grid_data):
#     for j,cell in enumerate(row):
#         grid[i,j] = cell
grid[0,0] = create_label("Parameter",color="#00BFD8")
grid[0,1] = create_label("Value",color="#00BFD8")
grid[0,2] = create_label("Units",color="#00BFD8")
## Magnetic Field
grid[1,0] = create_label("B - Magnetic Field Magnitude:")
grid[1,1],grid[1,2] = create_widget(FloatBox,property_name="B",unit=units.T,opts=[units.T,units.G,units.uG])

## Row 2 - Density Number
grid[2,0] = create_label("_"*20,color="#00BFD8")
grid[2,1] = create_label("Density Number")
grid[2,2] = create_label("_"*20,color="#00BFD8")

grid[3,0] = create_label("n - Standard Density Number:")
grid[3,1],grid[3,2] = create_widget(FloatBox,property_name="n",unit=units.m**-3,
opts=[units.m**-3,units.cm**-3,units.mm**-3])

grid[4,0] = create_label("n<sub>e</sub> - Electron Density Number:")
grid[4,1],grid[4,2] = create_widget(FloatBox,property_name="n_e",unit=units.m**-3,
opts=[units.m**-3,units.cm**-3,units.mm**-3])

grid[5,0] = create_label("n<sub>i</sub> - Ion Density Number:")
grid[5,1],grid[5,2] = create_widget(FloatBox,property_name="n_i",unit=units.m**-3,
opts=[units.m**-3,units.cm**-3,units.mm**-3])


## Temperature
grid[6,0] = create_label("_"*20,color="#00BFD8")
grid[6,1] = create_label("Temperature")
grid[6,2] = create_label("_"*20,color="#00BFD8")


grid[7,0] = create_label("T - Standard Temperature:")
grid[7,1] = create_widget(FloatBox,property_name="T",min=0,unit=units.K)
grid[7,2] = create_label("K",color="#a9a9a9")

grid[8,0] = create_label("T<sub>e</sub> - Electron Temperature:")
grid[8,1] = create_widget(FloatBox,property_name="T_e",min=0,unit=units.K)
grid[8,2] = create_label("K",color="#a9a9a9")

grid[9,0] = create_label("T<sub>i</sub> - Ion Temperature:")
grid[9,1] = create_widget(FloatBox,property_name="T_i",min=0,unit=units.K)
grid[9,2] = create_label("K",color="#a9a9a9")
## Row 4 - Particle
grid[10,0] = create_label("Particle:")
grid[10,1] = create_widget(ParticleBox,property_name="particle",property_alias="particle_type",placeholder="Enter particle for e.g. neutron,He 2+")

grid[11,0] = create_label("Convert to Hertz:")
grid[11,1] = create_widget(CheckBox,property_name="to_hz")

calculate_button = widgets.Button(description="Calculate Properties",button_style="info")
calculate_button.on_click(handle_button_click)
grid[-1,0] = calculate_button
display(grid)
# calculate_button = widgets.Button(description="Calculate Properties",button_style="info")
# calculate_button.on_click(handle_button_click)
# grid[-1,0] = calculate_button
# display(grid)

GridspecLayout(children=(HTML(value="<h3 style='margin:0px;color:#00BFD8'>Parameter<h3>", layout=Layout(grid_a…

GridspecLayout(children=(HTML(value="<h3 style='margin:0px;color:#00BFD8'>Parameter<h3>", layout=Layout(grid_a…

In [35]:
app

In [36]:
values_container

{}